## Using OpenAI API
Simple example demonstrating LLM function calling mechanism.
This notebook shows the raw API response including tool calling decisions.


In [1]:
import json
import os
from openai import OpenAI

In [4]:
# Load API key from SECRETS file
def load_secrets():
    """Load secrets from SECRETS file."""
    secrets = {}
    secrets_path = ('SECRETS')
    
    if not os.path.exists(secrets_path):
        raise FileNotFoundError(
            f"SECRETS file not found at {secrets_path}. "
            "Please create it based on SECRETS_example format."
        )
    
    with open(secrets_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                secrets[key.strip()] = value.strip()
    
    return secrets

In [5]:
# Load secrets and get API key
secrets = load_secrets()
api_key = secrets.get('OPENAI_API_KEY')

if not api_key:
    raise ValueError(
        "OPENAI_API_KEY not found in SECRETS file. "
        "Please add your API key to the SECRETS file following the SECRETS_example format."
    )

In [6]:
# Initialize the client with API key from SECRETS file
client = OpenAI(api_key=api_key)

In [7]:
# Define two simple functions that the LLM can call
def get_weather(location: str, unit: str = "celsius") -> str:
    """Get the current weather in a given location.
    
    Args:
        location: The city and state, e.g. San Francisco, CA
        unit: The unit of temperature (celsius or fahrenheit)
    
    Returns:
        A string describing the weather
    """
    # This is a mock function - in reality, you'd call a weather API
    return f"The weather in {location} is 22 degrees {unit} and sunny."


def calculate(expression: str) -> str:
    """Evaluate a mathematical expression.
    
    Args:
        expression: A mathematical expression to evaluate, e.g. "2 + 2"
    
    Returns:
        The result of the calculation
    """
    try:
        result = eval(expression)  # In production, use a safer evaluator
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error calculating {expression}: {str(e)}"


In [8]:
# Define the tools/functions schema for the API
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The unit of temperature"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Evaluate a mathematical expression",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "A mathematical expression to evaluate, e.g. '2 + 2'"
                    }
                },
                "required": ["expression"]
            }
        }
    }
]

In [34]:
#user_message = "Could you tell me a joke"
user_message = "Could tell me if it will rain tomorrow in Toulouse?"


In [35]:
# Make the API call with function calling enabled
response = client.chat.completions.create(
    model="gpt-4o-mini",  # or "gpt-4", "gpt-3.5-turbo", etc.
    messages=[
        {"role": "user", "content": user_message}
    ],
    tools=tools,
    tool_choice="auto"  # Let the model decide whether to use tools
)

In [36]:
response

ChatCompletion(id='chatcmpl-CgT422m1DOlSwXuDHV2ntEYKi7dut', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_vBmSSmOvoQKAVWpy8dkoATrk', function=Function(arguments='{"location":"Toulouse, France"}', name='get_weather'), type='function')]))], created=1764237534, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_50906f2aac', usage=CompletionUsage(completion_tokens=18, prompt_tokens=121, total_tokens=139, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [37]:
response.model_dump()

{'id': 'chatcmpl-CgT422m1DOlSwXuDHV2ntEYKi7dut',
 'choices': [{'finish_reason': 'tool_calls',
   'index': 0,
   'logprobs': None,
   'message': {'content': None,
    'refusal': None,
    'role': 'assistant',
    'annotations': [],
    'audio': None,
    'function_call': None,
    'tool_calls': [{'id': 'call_vBmSSmOvoQKAVWpy8dkoATrk',
      'function': {'arguments': '{"location":"Toulouse, France"}',
       'name': 'get_weather'},
      'type': 'function'}]}}],
 'created': 1764237534,
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_50906f2aac',
 'usage': {'completion_tokens': 18,
  'prompt_tokens': 121,
  'total_tokens': 139,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}}

In [38]:
response.choices

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_vBmSSmOvoQKAVWpy8dkoATrk', function=Function(arguments='{"location":"Toulouse, France"}', name='get_weather'), type='function')]))]

In [39]:
response.choices[0]

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_vBmSSmOvoQKAVWpy8dkoATrk', function=Function(arguments='{"location":"Toulouse, France"}', name='get_weather'), type='function')]))

In [40]:
response.choices[0].message

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_vBmSSmOvoQKAVWpy8dkoATrk', function=Function(arguments='{"location":"Toulouse, France"}', name='get_weather'), type='function')])

In [42]:
response.choices[0].message.function_call

In [45]:
message = response.choices[0].message
print(f"Role: {message.role}")
print(f"Content: {message.content}")
print(f"Tool Calls: {len(message.tool_calls) if message.tool_calls else 0}")

Role: assistant
Content: None
Tool Calls: 1


In [ ]:
# If tools were called, execute them
    if message.tool_calls:
        tool_results = []
        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            print(f"\nExecuting: {function_name}({function_args})")
            
            if function_name == "get_weather":
                result = get_weather(**function_args)
            elif function_name == "calculate":
                result = calculate(**function_args)
            else:
                result = f"Unknown function: {function_name}"
            
            print(f"Result: {result}")
            
            tool_results.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": result
            })
        
        # Make a follow-up call with the tool results
        print("\n" + "=" * 60)
        print("FOLLOW-UP API CALL WITH TOOL RESULTS:")
        print("=" * 60)